In [1]:
!pip3 install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 53.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=841315f7cded11fd791a89e7951b8831a84915b2420b1d5113f71ec7fd75d329
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
spark = pyspark.sql.SparkSession.builder\
        .master('local[*]')\
        .appName('vikram_midterm_Q2')\
        .getOrCreate()

In [3]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc

<SparkContext master=local[*] appName=vikram_midterm_Q2>

#Unigrams

In [37]:
text = sc.textFile("romeo-juliet-pg1777.txt")
# note as text file

In [38]:
import re
def func(text):
    text = text.lower()
    text = re.sub("[^0-9a-zA-Z]", " ", text)
    return text
text1 = text.map(func)
#function similar to assignment 1 
# converted to lower and replaced all special characters

In [39]:
text2 = text1.flatMap(lambda l: l.split(" "))
# split on white spaces

In [40]:
from operator import add
count = text2.map(lambda w: (w, 1)).reduceByKey(add)
for i in count.take(10): print(i)

('', 22916)
('this', 258)
('is', 389)
('presented', 1)
('project', 27)
('gutenberg', 24)
('in', 329)
('cooperation', 1)
('world', 32)
('library', 16)


In [41]:
final = count.toDF(schema=['word', 'count']).coalesce(8)
final.show(10)
# total count is 22916

+-----------+-----+
|       word|count|
+-----------+-----+
|           |22916|
|       this|  258|
|         is|  389|
|  presented|    1|
|    project|   27|
|  gutenberg|   24|
|         in|  329|
|cooperation|    1|
|      world|   32|
|    library|   16|
+-----------+-----+
only showing top 10 rows



In [42]:
final=final.filter(final.word!="")
final = final.withColumnRenamed('count', 'divisor')
final.show(10)
# final answer

+-----------+-------+
|       word|divisor|
+-----------+-------+
|       this|    258|
|         is|    389|
|  presented|      1|
|    project|     27|
|  gutenberg|     24|
|         in|    329|
|cooperation|      1|
|      world|     32|
|    library|     16|
|        inc|     11|
+-----------+-------+
only showing top 10 rows



#Bigrams

In [30]:
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_replace, upper,lower, col
sentences = sc.textFile("romeo-juliet-pg1777.txt")
mapper_bi = sentences.map(lambda s: s.split(" ")).flatMap(lambda s: [((s[i], s[i+1]), 1) for i in range(0, len(s)-1)])
#creating mapper to make bigrams
bigrams = mapper_bi.reduceByKey(lambda x, y :x+y)
#creating bigrams by reducing to the keys
bigrams = bigrams.toDF()
# Saving to dataframe

In [31]:

final_file = bigrams.withColumn("first_word", lower(regexp_replace(F.col("_1").getItem("_1"), "[^a-zA-Z0-9]", ""))).withColumn("second_word", lower(regexp_replace(F.col("_1").getItem("_2"), "[^a-zA-Z0-9]", ""))).withColumn("count", F.col("_2"))
# Lowering all the alphabets abnd replacing special characters
final_file = final_file.filter(F.col("first_word") != " ")
final_file = final_file.filter(F.col("second_word") != " ")
# Filtering to remove all the empty locations
final_file = final_file.select("first_word", "second_word", "count")

In [33]:
final_file.show()

+----------+------------+-----+
|first_word| second_word|count|
+----------+------------+-----+
|        is|   presented|    1|
|   project|   gutenberg|    1|
| gutenberg|          in|    1|
|     world|     library|    1|
|   library|         inc|    1|
|      from|       their|    4|
|     their|     library|    1|
|          |     project|    1|
| gutenberg|       often|    1|
|     often|    releases|    1|
|    etexts|        that|    1|
|    placed|          in|    1|
|    public|      domain|    1|
|      this|       etext|    1|
|   certain|   copyright|    1|
| copyright|implications|    1|
|       you|      should|    2|
|electronic|     version|    9|
|   version|          of|    9|
|       the|    complete|    9|
+----------+------------+-----+
only showing top 20 rows



In [46]:
finder = final_file.join(final, final_file.first_word == final.word, 'left').dropna()
# Left join on the dataframe

In [47]:
finder.show()

+----------+-------------+-----+---------+-------+
|first_word|  second_word|count|     word|divisor|
+----------+-------------+-----+---------+-------+
|     those|          who|    1|    those|     17|
|  carnegie|       mellon|    8| carnegie|     10|
|     those|     intended|    1|    those|     17|
|      some|       states|    1|     some|     58|
|  carnegie|       mellon|    1| carnegie|     10|
|       art|          not|    4|      art|     55|
|     those|           so|    1|    those|     17|
|     those|      persons|    2|    those|     17|
|     cures|         with|    1|    cures|      1|
|      some|         that|    1|     some|     58|
|       art|          dun|    1|      art|     55|
|      some|  consequence|    1|     some|     58|
|      some|         vile|    1|     some|     58|
|      some|fiveandtwenty|    1|     some|     58|
| solemnity|         this|    1|solemnity|      3|
|     those|        stars|    1|    those|     17|
|       art|         thou|    4

In [56]:
import pandas as pd
finder = finder.toPandas()

In [57]:
finder['percent'] = finder['count']/ finder['divisor']

In [58]:
finder
# percent output of the MLE(maximum likelihood estimation) 

,first_word,second_word,count,word,divisor,percent
0,those,who,1,those,17,0.058824
1,carnegie,mellon,8,carnegie,10,0.800000
2,those,intended,1,those,17,0.058824
3,some,states,1,some,58,0.017241
4,carnegie,mellon,1,carnegie,10,0.100000
...,...,...,...,...,...,...
18089,shalt,continue,1,shalt,11,0.090909
18090,shalt,be,2,shalt,11,0.181818
18091,empty,boxes,1,empty,4,0.250000
18092,living,corse,1,living,5,0.200000
